# POISSON 

usar un modelo de Regresión de Poisson, que es más adecuado para predicciones de conteo como crímenes, y asegurarnos de que las predicciones sean no negativas.

In [5]:
import pandas as pd
from sklearn.linear_model import PoissonRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Cargar datos
df = pd.read_csv('https://raw.githubusercontent.com/CristopherGR/tfm/main/datos_finales.csv?token=GHSAT0AAAAAACSP4QHBX5XMJS4TB3ZBGCCMZSNO3IA')

# Convertir fecha a formato datetime
df['fecha_detencion'] = pd.to_datetime(df['fecha_detencion'])

# Extraer el año de la fecha de detención
df['anio'] = df['fecha_detencion'].dt.year

# Agrupar datos por provincia y año
df_grouped = df.groupby(['provincia', 'anio']).size().reset_index(name='crimenes')

# Filtrar los datos para incluir solo los años de 2019 a 2023
df_grouped = df_grouped[df_grouped['anio'].between(2019, 2023)]

# Preparar los datos para el modelo
X = df_grouped[['provincia', 'anio']]
y = df_grouped['crimenes']

# Codificación de variables categóricas
X = pd.get_dummies(X, columns=['provincia'], drop_first=True)

# Ajustar un modelo de regresión de Poisson
model = PoissonRegressor(max_iter=1000)
model.fit(X, y)

# Predicciones en los datos de entrenamiento
y_pred = model.predict(X)

# Cálculo de métricas de desempeño
mape = (abs((y - y_pred) / y).mean()) * 100
mae = mean_absolute_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f'METRICAS DE DESEMPEÑO PARA POISSON')
print(f'  >   MAPE: {mape}%')
print(f'  >   MAE: {mae}')
print(f'  >   R²: {r2}')

# Crear DataFrame para predicciones de 2024 y 2025
future_years = pd.DataFrame({
    'provincia': df_grouped['provincia'].unique().repeat(2),
    'anio': [2024, 2025] * len(df_grouped['provincia'].unique())
})

# Codificación de variables categóricas para años futuros
X_future = pd.get_dummies(future_years, columns=['provincia'], drop_first=True)
X_future = X_future.reindex(columns=X.columns, fill_value=0)  # Asegurarse de que las columnas coincidan

# Realizar predicciones
future_years['prediccion_crimenes'] = model.predict(X_future)

# Asegurarse de que las predicciones sean no negativas
future_years['prediccion_crimenes'] = future_years['prediccion_crimenes'].clip(lower=0)

# Reordenar columnas para el resultado final
resultados_finales = future_years[['provincia', 'anio', 'prediccion_crimenes']]

resultados_finales


METRICAS DE DESEMPEÑO PARA POISSON
  >   MAPE: 9.119955501651908%
  >   MAE: 277.1383441886693
  >   R²: 0.9869921284157344


c:\Users\crist\anaconda3\Lib\site-packages\sklearn\linear_model\_glm\glm.py:284: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


,provincia,anio,prediccion_crimenes
0,AZUAY,2024,2978.442257
1,AZUAY,2025,2978.169900
2,BOLIVAR,2024,854.252390
3,BOLIVAR,2025,854.174275
4,CARCHI,2024,1911.147137
5,CARCHI,2025,1910.972377
6,CAÑAR,2024,1396.194951
7,CAÑAR,2025,1396.067279
8,CHIMBORAZO,2024,2336.689030
9,CHIMBORAZO,2025,2336.475358


# SEGUNDA POISSON - AUMENTO ITERACIONES Y ESCALADO

Con base en la comparación de métricas y la naturaleza de los resultados, es más factible y recomendable utilizar los resultados de la última implementación. La combinación de Regresión de Poisson y el escalado de datos ha mejorado significativamente la precisión y confiabilidad del modelo.

Uso de Regresión de Poisson: Este modelo es más adecuado para datos de conteo y asegura que las predicciones sean no negativas.

In [6]:
import pandas as pd
from sklearn.linear_model import PoissonRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Cargar datos
df = pd.read_csv('https://raw.githubusercontent.com/CristopherGR/tfm/main/datos_finales.csv?token=GHSAT0AAAAAACSP4QHBX5XMJS4TB3ZBGCCMZSNO3IA')

# Convertir fecha a formato datetime
df['fecha_detencion'] = pd.to_datetime(df['fecha_detencion'])

# Extraer el año de la fecha de detención
df['anio'] = df['fecha_detencion'].dt.year

# Agrupar datos por provincia y año
df_grouped = df.groupby(['provincia', 'anio']).size().reset_index(name='crimenes')

# Filtrar los datos para incluir solo los años de 2019 a 2023
df_grouped = df_grouped[df_grouped['anio'].between(2019, 2023)]

# Preparar los datos para el modelo
X = df_grouped[['provincia', 'anio']]
y = df_grouped['crimenes']

# Codificación de variables categóricas
X = pd.get_dummies(X, columns=['provincia'], drop_first=True)

# Escalar los datos y mejorar la convergencia del algoritmo
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Ajustar un modelo de regresión de Poisson
model = PoissonRegressor(max_iter=2000)
model.fit(X_scaled, y)

# Predicciones en los datos de entrenamiento
y_pred = model.predict(X_scaled)

# Cálculo de métricas de desempeño
mape = (abs((y - y_pred) / y).mean()) * 100
mae = mean_absolute_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f'METRICAS DE DESEMPEÑO PARA POISSON')
print(f'  >   MAPE: {mape}%')
print(f'  >   MAE: {mae}')
print(f'  >   R²: {r2}')

# Crear DataFrame para predicciones de 2024 y 2025
future_years = pd.DataFrame({
    'provincia': df_grouped['provincia'].unique().repeat(2),
    'anio': [2024, 2025] * len(df_grouped['provincia'].unique())
})

# Codificación de variables categóricas para años futuros
X_future = pd.get_dummies(future_years, columns=['provincia'], drop_first=True)
X_future = X_future.reindex(columns=X.columns, fill_value=0)  # Asegurarse de que las columnas coincidan

# Escalar los datos futuros
X_future_scaled = scaler.transform(X_future)

# Realizar predicciones
future_years['prediccion_crimenes'] = model.predict(X_future_scaled)

# Asegurarse de que las predicciones sean no negativas
future_years['prediccion_crimenes'] = future_years['prediccion_crimenes'].clip(lower=0)

# Reordenar columnas para el resultado final
resultados_finales = future_years[['provincia', 'anio', 'prediccion_crimenes']]

resultados_finales


METRICAS DE DESEMPEÑO PARA POISSON
  >   MAPE: 7.472371908521265%
  >   MAE: 206.29334964904436
  >   R²: 0.9929138892854464


,provincia,anio,prediccion_crimenes
0,AZUAY,2024,2737.682987
1,AZUAY,2025,2624.444444
2,BOLIVAR,2024,727.502595
3,BOLIVAR,2025,697.410968
4,CARCHI,2024,1670.579651
5,CARCHI,2025,1601.479610
6,CAÑAR,2024,1212.855397
7,CAÑAR,2025,1162.688165
8,CHIMBORAZO,2024,2051.010939
9,CHIMBORAZO,2025,1966.175152


In [7]:
resultados_finales.to_csv('predicciones.csv', index=False)